In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("D:\Downloads\creditcard.csv.zip")

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def z_score_normalization(x):
    Mu = np.mean(x,axis = 0)
    std = np.std(x,axis = 0)
    x_norm = (x-Mu)/(std)

    return x_norm

Next thing is to handle the imbalance in the data that is in this the data have high no of non-fraudelent cases 
so inorder to solve this imabalance we nee to solve this 
1)undersampling
Reduce the number of non-fraud cases (Class 0) so it matches the number of fraud cases (Class 1).

✏️ Example:
From 284,315 non-frauds → randomly pick only 492 to match the 492 frauds.
So now you have:

492 frauds

492 non-frauds
2)oversampling
Duplicate or synthesize more fraud cases (Class 1) so it matches the number of non-fraud cases.

Two types:
🔁 a. Random Oversampling
Simply duplicate existing fraud samples

Easy but may lead to overfitting

🧠 b. SMOTE (Synthetic Minority Oversampling Technique)
Generates new synthetic fraud samples based on real ones

Creates slightly modified “fake” examples to enrich diversity

In [ ]:
# Going with undersampling

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
def compute_cost(x,y,w,b):
    x = np.array(x)
    y = np.array(y)
    m = x.shape[0]
    cost = 0 
    for i in range(m):
        z_i = np.dot(x[i],w)+b
        f_wb_i = sigmoid(z_i)
        cost+=-y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
    cost/=(m)
    return cost
        

In [ ]:
w_init = np.zeros(X_train_balanced.shape[1])
b_init = 0

In [ ]:
def compute_gradient_logistic(x,y,w,b):
    x = np.array(x)
    y = np.array(y)
    m,n = x.shape
    dj_dw = np.zeros((n,))
    dj_db = 0
    for i in range(m):
        f_wb_i = sigmoid(np.dot(x[i],w)+b)
        err_i  = (f_wb_i - y[i])
        for j in range(n):
            dj_dw+=err_i*x[i,j]

        dj_db+=err_i
    dj_dw = dj_dw/(m)
    dj_db = dj_db/(m)
    return dj_dw,dj_db

In [ ]:
import math

In [ ]:
def compute_gradient_descent(x,y,w,b,num_iterations,alpha):

    j_history = []
    for i in range(num_iterations):
        dj_dw,dj_db = compute_gradient_logistic(X_train_balanced,y_train_balanced,w,b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
        if i<1000000:
             j_history.append(compute_cost(X_train_balanced,y_train_balanced,w,b))
        if i% math.ceil(num_iterations / 10) == 0:
             print(f"Iteration {i:4d}: Cost {j_history[-1]}")
    return w,b,j_history

In [ ]:
def predict(x,w,b):
    z = np.dot(x,w)+b
    return sigmoid(z)>=0.9

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

new_df = df.copy()

new_df['time_bin'] = pd.cut(new_df['Time'],bins = np.arange(0,new_df['Time'].max()+3600,3600))

new_df = new_df[new_df['time_bin'].notna()]

avg_amount_per_time_bin = new_df.groupby('time_bin')['Amount'].mean()
avg_amount_per_time_bin.name = 'average_amount_per_time_bin'
new_df = new_df.join(avg_amount_per_time_bin,on = 'time_bin')


In [ ]:
new_df = new_df[new_df['average_amount_per_time_bin'].notna()]

In [ ]:
new_df['Amount_by_Avg'] = new_df['Amount']/new_df['average_amount_per_time_bin']
new_df['log_amount_by_avg'] = np.log1p(new_df['Amount_by_Avg'])


In [ ]:

new_df['log_Amount_by_Avg_scaled'] = z_score_normalization(new_df[['log_amount_by_avg']])

In [ ]:
new_df

In [ ]:
new_df.shape

In [ ]:
w_init = np.zeros(X_train_balanced.shape[1])
b_init = 0

In [ ]:

new_df.drop(["average_amount_per_time_bin"],axis=1)


In [ ]:
new_df

In [ ]:
new_df.shape[0]

In [ ]:
norm_amount = z_score_normalization(new_df[['Amount']])
norm_Time   = z_score_normalization(new_df[['Time']])

In [ ]:
new_df = new_df.join([norm_amount.rename(columns={'Amount': 'norm_amount'}),
                      norm_Time.rename(columns={'Time': 'norm_time'})])

In [ ]:
new_df = new_df.drop(['Time'],axis = 1)

In [ ]:
new_df = new_df.drop(['Amount'],axis = 1)

In [ ]:
new_df = new_df.drop(['average_amount_per_time_bin'],axis = 1)

In [ ]:
new_df

In [ ]:
x = new_df.drop('Class',axis = 1)
Y = new_df[['Class']]

In [ ]:
x

In [ ]:
Y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x,Y,test_size = 0.2,random_state = 42,stratify = Y)

In [ ]:
new_df.columns[new_df.isnull().any()]

In [ ]:
train_data_new = pd.concat([X_train,y_train],axis = 1)

In [ ]:
fraud_case_new = train_data_new[train_data_new["Class"]==1]

In [ ]:
non_fraud_case_new = train_data_new[train_data_new["Class"]==0]

In [ ]:
fraud_case_new

In [ ]:
non_fraud_case_new

In [ ]:
non_fraud_case_sampled = non_fraud_case_new.sample(n=len(fraud_case_new),random_state=42)

In [ ]:
count = 0
for i in range(len(fraud_case_new[fraud_case_new["Class"]==1])):
    count+=1
print(count)

In [ ]:
count = 0
for i in range(len(non_fraud_case_sampled[non_fraud_case_sampled["Class"]==0])):
    count+=1
print(count)

In [ ]:
balanced_data_new = pd.concat([fraud_case_new,non_fraud_case_sampled]) 

In [ ]:
balanced_data_new

In [ ]:
balanced_data_new = balanced_data_new.sample(frac = 1,random_state = 42).reset_index(drop = True)
X_train_balanced_new = balanced_data_new.drop('Class',axis = 1)
y_train_balanced_new = balanced_data_new[['Class']]

In [ ]:
X_train_balanced_new

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('time_bin',axis = 1)

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('Amount_by_Avg',axis = 1)

In [ ]:
X_train_balanced_new=X_train_balanced_new.drop('log_amount_by_avg',axis = 1)

In [ ]:
X_train_balanced_new

In [ ]:
y_train_balanced_new

In [ ]:
w_init_new = np.zeros(X_train_balanced_new.shape[1])
b_init_new = 0

In [ ]:
print(X_train_balanced_new.dtypes)

In [ ]:
def compute_gradient_descent(x,y,w,b,num_iterations,alpha):

    j_history = []
    for i in range(num_iterations):
        dj_dw,dj_db = compute_gradient_logistic(X_train_balanced_new,y_train_balanced_new,w,b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
        if i<1000000:
             j_history.append(compute_cost(X_train_balanced_new,y_train_balanced_new,w,b))
        if i% math.ceil(num_iterations / 10) == 0:
             print(f"Iteration {i:4d}: Cost {j_history[-1]}")
    return w,b,j_history

In [ ]:
num_iterations = 1000
alpha = 0.01
w,b,j_history = compute_gradient_descent(X_train_balanced_new,y_train_balanced_new,w_init_new,b_init_new,num_iterations,alpha)

In [ ]:
print(w,b)

In [ ]:
X_test

In [ ]:
X_test = X_test.drop(columns = ['time_bin','Amount_by_Avg','log_amount_by_avg'])


In [ ]:
X_test

In [ ]:
def predict_new(x,w,b):
    z = np.dot(x,w)+b
    return sigmoid(z)>=0.9

In [ ]:
y_pred = predict_new(X_test,w,b)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score

accuracy = accuracy_score(y_test,y_pred)

In [ ]:
print(accuracy)

In [ ]:
precision = precision_score(y_test,y_pred)

In [ ]:
print(precision)